<a href="https://colab.research.google.com/github/Czedros/CSE352-Machine-Learning-Assignments/blob/main/Knowledge-based/z3logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Name: Kay Zhang

I understand that my submission needs to be my own work: [add initials]

I understand that ChatGPT / Copilot / other AI tools are not allowed: [add initials]

Total Points: **75**

Complete this notebook and submit it (save/print as pdf). Make sure all output is correct in the pdf before submitting (it sometimes gets cut off). The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Z3 and Z3py resources

For this problem you will be solving constraint satisfaction and logic problems using the Z3 SMT solver via the z3py python library.

The demo code using z3py we went over in class is here: [link](https://drive.google.com/drive/folders/16HQXiwdcaman1IpC7H6vSJ71QQQ-aL2z?usp=sharing)

Online help for z3py is here: [link](https://ericpony.github.io/z3py-tutorial/guide-examples.htm)

## Installation

We first install to get the z3solver library using pip and test that works.

In [1]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 58.1 MB/s eta 0:00:00


In [2]:
# Run the first example from the z3py guide as a test: https://ericpony.github.io/z3py-tutorial/guide-examples.htm
from z3 import *

x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


# Sudoku as a constraint satisfaction problem (CSP)

Sudoku is a popular number-placement puzzle that originated in France in the end of the 19th century.  Modern Sudoku was likely invented by Howard Garns from Connersville, Indiana and was first published in 1979 under the name *Number Place*. The objective of the puzzle is to place numbers 1-9 on a 9$\times$9 grid, such that each number occurs only once in every row, every column, and every of the nine 3$\times$3 sub-grids that compose the main grid.
Sudoku puzzles are grids that have been partially occupied with numbers. The task is then to occupy the remaining fields in such a way that the constraints on rows, columns, and sub-grids are satisfied.
For more information about Sudoku refer to its Wikipedia page at http://en.wikipedia.org/wiki/Sudoku.

This problem has two parts. In the first part, you will write the boolean constraints in mathematical notation for solving a Sudoku puzzle. In the second part, you will write code and invoke the Z3 solver to solve the Sudoku instance and answer various questions about the solution.

## Part 1 (Constraints)
In text, define and write constraints over **boolean** variables corresponding to each number being in each cell. For example, you may use $X_{i,j}^k$ as teh variable that is true if and only if the number $k$ is in row $i$ and column $j$ (the variable is true if $k$ is in cell $(i, j)$ and false otherwise). Now write the following boolean constaints over these variables:

* Write the boolean formula for the constraints that the number 5 can occur at most once in the first row. **(3 point)**
* Write the boolean formula for the constraints that the number 6 can occur at most once in the first column. **(3 point)**
* Write the boolean formula for the constraints that the number 9 can occur at most once in the top left 3$\times$3 sub-grid. **(4 point)**

Solution:

Let $X_{i,j}^k$ be a boolean variable such that

$X_{i,j}^k$ = true if cell(i,j) contains number k

and

$X_{i,j}^k$ = false otherwise

1.
$\bigwedge_{1 \leq j < l \leq 9} \neg (X_{1,j}^5 \land X_{1,l}^5)$

2.
$\bigwedge_{1 \leq i < m \leq 9} \neg (X_{i,1}^6 \land X_{m,1}^6)$

3.$\bigwedge_{(i_1, j_1) < (i_2, j_2)} \neg \left( X_{i_1, j_1}^9 \land X_{i_2, j_2}^9 \right)$


## Part 2 (Coding)

Encode the above constraints and all the other ones as a CSP using z3py and solve the Sudoku instance given in the figure below. **Use only boolean variables** and do not use the **Distinct** function.

* Provide the code to solve that solves the given problem instance using z3py and only boolean variables. The code should output a reasonable visualization of the solution, for example printed in text. **(10 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/sudoku1.jpg" width="400"/>



In [1]:
# z3py code to solve above Sudoku goes here. Do not use Int(*), only Bool(*)
from z3 import *

# Initialize the solver
solver = Solver()

# Variables: X[i][j][k] is True if cell (i,j) contains number k+1
X = [[[Bool(f"X_{i}_{j}_{k}") for k in range(9)] for j in range(9)] for i in range(9)]

# 1. Each cell contains exactly one number
for i in range(9):
    for j in range(9):
        # At least one number in each cell
        solver.add(Or(X[i][j]))
        # At most one number in each cell
        for k1 in range(9):
            for k2 in range(k1 + 1, 9):
                solver.add(Or(Not(X[i][j][k1]), Not(X[i][j][k2])))

# 2. Each number appears at most once in each row
for i in range(9):
    for k in range(9):
        for j1 in range(9):
            for j2 in range(j1 + 1, 9):
                solver.add(Or(Not(X[i][j1][k]), Not(X[i][j2][k])))

# 3. Each number appears at most once in each column
for j in range(9):
    for k in range(9):
        for i1 in range(9):
            for i2 in range(i1 + 1, 9):
                solver.add(Or(Not(X[i1][j][k]), Not(X[i2][j][k])))

# 4. Each number appears at most once in each 3x3 block
for bi in range(3):
    for bj in range(3):
        for k in range(9):
            block_cells = [(bi*3 + di, bj*3 + dj) for di in range(3) for dj in range(3)]
            for idx1 in range(9):
                for idx2 in range(idx1 + 1, 9):
                    i1, j1 = block_cells[idx1]
                    i2, j2 = block_cells[idx2]
                    solver.add(Or(Not(X[i1][j1][k]), Not(X[i2][j2][k])))

# 5. Add known values from the puzzle (from image)
# Given values (1-based) from the image
puzzle = [
    [0, 1, 0, 4, 0, 2, 0, 5, 0],
    [5, 0, 0, 0, 0, 0, 0, 0, 6],
    [0, 0, 0, 3, 0, 1, 0, 0, 0],
    [7, 0, 5, 0, 0, 0, 4, 0, 8],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [2, 0, 8, 0, 0, 0, 5, 0, 9],
    [0, 0, 0, 9, 0, 6, 0, 0, 0],
    [6, 0, 0, 0, 0, 0, 0, 0, 2],
    [0, 7, 0, 1, 0, 3, 0, 4, 0],
]

for i in range(9):
    for j in range(9):
        if puzzle[i][j] != 0:
            k = puzzle[i][j] - 1
            solver.add(X[i][j][k])

def print_grid(model, X):
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-" * 21)
        row = ""
        for j in range(9):
            if j % 3 == 0 and j != 0:
                row += "| "
            for k in range(9):
                if is_true(model.evaluate(X[i][j][k])):
                    row += str(k + 1) + " "
        print(row.strip())

if solver.check() == sat:
    model = solver.model()
    print_grid(model, X)
else:
    print("No solution found.")



3 1 9 | 4 6 2 | 8 5 7
5 2 4 | 7 9 8 | 1 3 6
8 6 7 | 3 5 1 | 9 2 4
---------------------
7 3 5 | 2 1 9 | 4 6 8
1 9 6 | 8 4 5 | 2 7 3
2 4 8 | 6 3 7 | 5 1 9
---------------------
4 5 3 | 9 2 6 | 7 8 1
6 8 1 | 5 7 4 | 3 9 2
9 7 2 | 1 8 3 | 6 4 5


### Uniqueness
Is your solution unique? Prove it with a Z3py solver or provide a second solution. **(5 points)**

In [2]:
# Save first model
if solver.check() == sat:
    model1 = solver.model()

    # Store current solution in a list
    solution = [[-1 for _ in range(9)] for _ in range(9)]
    for i in range(9):
        for j in range(9):
            for k in range(9):
                if is_true(model1.evaluate(X[i][j][k])):
                    solution[i][j] = k  # Store index (i.e., value - 1)

    # Create a new constraint: there exists at least one cell different
    diff_constraint = []
    for i in range(9):
        for j in range(9):
            current_k = solution[i][j]
            # At least one cell must differ
            diff_constraint.append(Not(X[i][j][current_k]))

    # Push solver state and add the negation of current solution
    solver.push()
    solver.add(Or(diff_constraint))  # At least one cell differs
    result = solver.check()

    if result == unsat:
        print("✅ The solution is UNIQUE.")
    else:
        print("❗ There is ANOTHER solution.")
        # Optionally print the alternative
        model2 = solver.model()
        print("Another possible solution:")
        print_grid(model2, X)

    solver.pop()
else:
    print("No solution found.")

✅ The solution is UNIQUE.


### More Uniqueness
If you delete the 1 in the top left box of the Sudoku problem above, how many unique solutions are there? Hint: should be fewer than 5000. **(5 points)**

In [ ]:
# z3py code to solve above Sudoku goes here. Do not use Int(*), only Bool(*)
from z3 import *

solver = Solver()
X = [[[Bool(f"X_{i}_{j}_{k}") for k in range(9)] for j in range(9)] for i in range(9)]


def count_all_solutions(X, base_solver):
    count = 0
    solver = Solver()
    solver.add(base_solver.assertions())  # Clone constraints

    while solver.check() == sat:
        model = solver.model()
        count += 1

        # Build blocking clause to avoid this exact solution
        blocking_clause = []
        for i in range(9):
            for j in range(9):
                for k in range(9):
                    lit = X[i][j][k]
                    if is_true(model.evaluate(lit)):
                        # This variable was True in model → block it
                        blocking_clause.append(Not(lit))
        solver.add(Or(blocking_clause))  # At least one must change

    return count



# 1. Each cell contains exactly one number
for i in range(9):
    for j in range(9):
        # At least one number in each cell
        solver.add(Or(X[i][j]))
        # At most one number in each cell
        for k1 in range(9):
            for k2 in range(k1 + 1, 9):
                solver.add(Or(Not(X[i][j][k1]), Not(X[i][j][k2])))

# 2. Each number appears at most once in each row
for i in range(9):
    for k in range(9):
        for j1 in range(9):
            for j2 in range(j1 + 1, 9):
                solver.add(Or(Not(X[i][j1][k]), Not(X[i][j2][k])))

# 3. Each number appears at most once in each column
for j in range(9):
    for k in range(9):
        for i1 in range(9):
            for i2 in range(i1 + 1, 9):
                solver.add(Or(Not(X[i1][j][k]), Not(X[i2][j][k])))

# 4. Each number appears at most once in each 3x3 block
for bi in range(3):
    for bj in range(3):
        for k in range(9):
            block_cells = [(bi*3 + di, bj*3 + dj) for di in range(3) for dj in range(3)]
            for idx1 in range(9):
                for idx2 in range(idx1 + 1, 9):
                    i1, j1 = block_cells[idx1]
                    i2, j2 = block_cells[idx2]
                    solver.add(Or(Not(X[i1][j1][k]), Not(X[i2][j2][k])))

# 5. Add known values from the puzzle (from image)
# Given values (1-based) from the image
puzzle = [
    [0, 0, 0, 4, 0, 2, 0, 5, 0],
    [5, 0, 0, 0, 0, 0, 0, 0, 6],
    [0, 0, 0, 3, 0, 1, 0, 0, 0],
    [7, 0, 5, 0, 0, 0, 4, 0, 8],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [2, 0, 8, 0, 0, 0, 5, 0, 9],
    [0, 0, 0, 9, 0, 6, 0, 0, 0],
    [6, 0, 0, 0, 0, 0, 0, 0, 2],
    [0, 7, 0, 1, 0, 3, 0, 4, 0],
]

total_solutions = count_all_solutions(X, solver)
print(f"🔢 Total number of solutions: {total_solutions}")

# Miracle Sudoku

Using z3py and only boolean variables, solve the Miracle Sudoku ([spoiler](https://www.youtube.com/watch?v=yKf9aUIxdb4)).
In this puzzle normal Sudoku rules apply in addition to the following rules:
* Any two cells separated by a knight's move from chess (moving forward two boxes and over one, in any direction) cannot contain the same digit (see image below).
* Any two cells separated by a king's move from chess (one box in any direction including diagonals) cannot contain the same digit.
* Any two orthogonally adjacent (up, down, left, or right) cells cannot contain consecutive digits.

### Knight's move constraint image
<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/knights.jpg" width="400"/>

The knight's move constraint would mean the yellow boxes cannot contain a 1.


## Miracle Sudoku Starting Information
Provide the code to solve the puzzle given below. The code should output a reasonable visualization of the solution, for example using text. **(10 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/miracle.jpg" width="400"/>

In [ ]:
# Solution code for Miracle Sudoku using z3py
from z3 import *

from z3 import *

# Setup: Bool vars for 9x9 grid, digits 1–9
X = [[[Bool(f"X_{i}_{j}_{k}") for k in range(9)] for j in range(9)] for i in range(9)]

solver = Solver()

# Rule 1: Each cell has exactly one digit
for i in range(9):
    for j in range(9):
        solver.add(Or(X[i][j]))
        for k1 in range(9):
            for k2 in range(k1 + 1, 9):
                solver.add(Or(Not(X[i][j][k1]), Not(X[i][j][k2])))

# Rule 2: Each digit appears once per row and column
for k in range(9):
    for i in range(9):
        for j1 in range(9):
            for j2 in range(j1 + 1, 9):
                solver.add(Or(Not(X[i][j1][k]), Not(X[i][j2][k])))
    for j in range(9):
        for i1 in range(9):
            for i2 in range(i1 + 1, 9):
                solver.add(Or(Not(X[i1][j][k]), Not(X[i2][j][k])))

# Rule 3: Each 3x3 box has unique digits
for k in range(9):
    for bi in range(3):
        for bj in range(3):
            block = [(bi * 3 + dx, bj * 3 + dy) for dx in range(3) for dy in range(3)]
            for m in range(9):
                for n in range(m + 1, 9):
                    i1, j1 = block[m]
                    i2, j2 = block[n]
                    solver.add(Or(Not(X[i1][j1][k]), Not(X[i2][j2][k])))

# Miracle rule 1: No knight move duplicates
knight_offsets = [(-2, -1), (-1, -2), (-2, 1), (-1, 2),
                  (1, -2), (2, -1), (2, 1), (1, 2)]

for i in range(9):
    for j in range(9):
        for dx, dy in knight_offsets:
            ni, nj = i + dx, j + dy
            if 0 <= ni < 9 and 0 <= nj < 9:
                for k in range(9):
                    solver.add(Or(Not(X[i][j][k]), Not(X[ni][nj][k])))

# Miracle rule 2: No king move duplicates
king_offsets = [(-1, -1), (-1, 0), (-1, 1),
                ( 0, -1),          ( 0, 1),
                ( 1, -1), ( 1, 0), ( 1, 1)]

for i in range(9):
    for j in range(9):
        for dx, dy in king_offsets:
            ni, nj = i + dx, j + dy
            if 0 <= ni < 9 and 0 <= nj < 9:
                for k in range(9):
                    solver.add(Or(Not(X[i][j][k]), Not(X[ni][nj][k])))

# Miracle rule 3: No consecutive orthogonal neighbors
orthogonal_offsets = [(-1, 0), (1, 0), (0, -1), (0, 1)]
for i in range(9):
    for j in range(9):
        for dx, dy in orthogonal_offsets:
            ni, nj = i + dx, j + dy
            if 0 <= ni < 9 and 0 <= nj < 9:
                for k in range(9):
                    if k > 0:
                        solver.add(Or(Not(X[i][j][k]), Not(X[ni][nj][k - 1])))
                    if k < 8:
                        solver.add(Or(Not(X[i][j][k]), Not(X[ni][nj][k + 1])))

# Initial clue (from the Miracle Sudoku): Only one value is given
# Source: [video](https://www.youtube.com/watch?v=yKf9aUIxdb4)
# The original puzzle gives a '1' at position (6, 2) [0-indexed: (5, 1)]
solver.add(X[5][1][0])  # 1 is index 0

# Solve
if solver.check() == sat:
    model = solver.model()
    for i in range(9):
        row = ""
        for j in range(9):
            for k in range(9):
                if is_true(model.evaluate(X[i][j][k])):
                    row += str(k + 1) + " "
                    break
        print(row.strip())
else:
    print("No solution found.")


---
# Number Challenge in Z3py

Write Z3py code to solve the Number Challenge puzzle shown below and print the solution found. You can use the `Int` type to create integer variables in z3py. **(20 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/chal.jpg" width="300"/>

In [ ]:
# Write your solution code for Number Challenge using z3py
from z3 import *

# example using Int type (feel free to delete)
s = Solver()

x = Int('x')
y = Int('y')

s.add((2*y) + x == 20)
s.add(x - y == 2)

if s.check() == sat:
    print("sat")
    m = s.model()
    print(f"model: {m}")

    print(f"x: {x}")
    print(f"y: {y}")
else:
    print("unsat")

sat
model: [y = 6, x = 8]
x: x
y: y





---


# Propositional Logic

Next, we will use truth tables and z3py to prove logical entailment.

## Truth Tables

Using markdown tables and bold to show relevant rows, use the truth table method to show logical truths.


### Example

Here's an example that shows $p, p \rightarrow q \models q$.  

| p | q | p -> q |
|-------|-------|---------------------|
| **T**     | **T**     | **T**                   |
| T     | F     | F                   |
| F     | T     | T                   |
| F     | F     | T                   |

Since $q$ is true whenever $p$ and $p \rightarrow q$ are true, the expression is true.

### Questions

Prove that the following semantic entailment expressions are true by using the truth table approach.  Bold the appropriate rows **(7 point)**

 * $(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s, t \rightarrow q \models r$
 * $\emptyset \models q \rightarrow (p \rightarrow (p \rightarrow (q \rightarrow p)))$

(add markdown tables here)



---


 ## Logical Entailment using z3py

 Use Z3py to prove the above two entailments are true using satisfiability. See section 7.5 in the book for details. **(8 points)**

In [ ]:
# answer code here